In [1]:
# Copyright 2022 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions anda
# limitations under the License.
# ==============================================================================

<img src="https://developer.download.nvidia.com/notebooks/dlsw-notebooks/merlin_models-transformers-net-item-prediction/nvidia_logo.png" style="width: 90px; float: right;">

# Transformer-based architecture for next-item prediction task

## Overview

In this use case we will train a Transformer-based architecture for next-item prediction task.

We will use the [booking.com dataset](https://github.com/bookingcom/ml-dataset-mdt) to train a session-based model. The dataset contains 1,166,835 of anonymized hotel reservations in the train set and 378,667 in the test set. Each reservation is a part of a customer's trip (identified by `utrip_id`) which includes consecutive reservations.

We will reshape the data to organize it into 'sessions'. Each session will be a full customer itinerary in chronological order. The goal will be to predict the city_id of the final reservation of each trip.


### Learning objectives

- Training a Transformer-based architecture for next-item prediction task

## Downloading and preparing the dataset

We will download the dataset using a functionality provided by merlin models. The dataset can be found on GitHub [here](https://github.com/bookingcom/ml-dataset-mdt).

In [5]:


# !cd /models && git pull origin main && pip install .
# # cd /nvtabular && git pull && pip install .
# # cd /core && git pull && pip install .
# # cd /systems && git pull && pip install .

From https://github.com/NVIDIA-Merlin/Models
 * branch                main       -> FETCH_HEAD
Updating 8ac9090dc..ceed7c5b9
Fast-forward
 .github/workflows/docs-preview-pr.yaml             |  118 +-
 .github/workflows/docs-sched-rebuild.yaml          |   24 +-
 .pre-commit-config.yaml                            |    1 +
 Makefile                                           |    2 +-
 ci/test_unit.sh                                    |    2 +-
 docs/source/conf.py                                |    4 +
 ...2-Merlin-Models-and-NVTabular-integration.ipynb |  238 +-
 examples/03-Exploring-different-models.ipynb       |  463 +-
 examples/04-Exporting-ranking-models.ipynb         |  165 +-
 examples/05-Retrieval-Model.ipynb                  |  893 +--
 ...-your-own-architecture-with-Merlin-Models.ipynb |  509 +-
 ...ing-of-large-embedding-tables-by-LazyAdam.ipynb |   11 +-
 .../entertainment-with-pretrained-embeddings.ipynb |   74 +-
 .../incremental-training-with-layer-freezing.ipynb |  91

Processing /models
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


  Created wheel for merlin-models: filename=merlin_models-0.9.0+44.gceed7c5b9-py3-none-any.whl size=364967 sha256=a90747c5808af4158496f1ec8323c7dd845d22638135eea35ce95580b82bd5e1
  Stored in directory: /tmp/pip-ephem-wheel-cache-fp9i6g7q/wheels/4d/e8/98/0493db55fff90dc9af123f55a9455b96f7f8166c912a02c8a6
Successfully built merlin-models
  Attempting uninstall: merlin-models
    Found existing installation: merlin-models 0.9.0
    Uninstalling merlin-models-0.9.0:
      Successfully uninstalled merlin-models-0.9.0


In [1]:
from merlin.core.dispatch import get_lib
from merlin.datasets.ecommerce import get_booking
import numpy as np

from nvtabular import *
from nvtabular import ops
from merlin.models.tf import Loader
from merlin.schema.tags import Tags

get_booking('/workspace/data')
train = get_lib().read_csv('/workspace/data/train_set.csv', parse_dates=['checkin', 'checkout'])

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2022-11-16 02:14:54.749335: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:991] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-16 02:14:54.749755: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:991] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-16 02:14:54.749957: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:991] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-16 02:14:55.039058: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is opti

Each reservation has a unique `utrip_id`. During each trip a customer vists several destinations.

In [2]:
train.head()

user_id    checkin   checkout  city_id device_class  affiliate_id  \
0  1000027 2016-08-13 2016-08-14     8183      desktop          7168   
1  1000027 2016-08-14 2016-08-16    15626      desktop          7168   
2  1000027 2016-08-16 2016-08-18    60902      desktop          7168   
3  1000027 2016-08-18 2016-08-21    30628      desktop           253   
4  1000033 2016-04-09 2016-04-11    38677       mobile           359   

  booker_country hotel_country   utrip_id  
0        Elbonia        Gondal  1000027_1  
1        Elbonia        Gondal  1000027_1  
2        Elbonia        Gondal  1000027_1  
3        Elbonia        Gondal  1000027_1  
4         Gondal  Cobra Island  1000033_1

We will train on sequences of `city_id` and `booker_country` and based on this information, our model will attempt to predict the next `city_id` (the next hop in the journey).

We will train a transformer model that can work with sequences of variable length within a batch. This functionality is provided to us out of the box and doesn't require any changes to the architecture. Thanks to it we do not have to pad or trim our sequences to any particular length -- our model can make effective use of all of the data!

With one exception. For a masked language model that we will be training, we need to discard sequences that are shorter than two hops. This makes sense as there is nothing our model could learn if it was only presented with an itinerary with a single destination on it!

Let us begin by splitting the data into a train and validation set based on trip ID.

Let's see how many unique trips there are in the dataset. Also, let us shuffle the trips along the way so that our validation set consists of a random sample of our train set.

In [3]:
utrip_ids = train.sample(frac=1).utrip_id.unique()
len(utrip_ids)

217686

Now let's assign data to our train and validation sets. Furthermore, we sort the data by `utrip_id` and `checkin`. This way we ensure our sequences of visited `city_ids` will be in proper order!

In [4]:
train_set_utrip_ids = utrip_ids[:int(0.8 * utrip_ids.shape[0])]
validation_set_utrip_ids = utrip_ids[int(0.8 * utrip_ids.shape[0]):]

train_set = train[train.utrip_id.isin(train_set_utrip_ids)].sort_values(['utrip_id', 'checkin'])
validation_set = train[train.utrip_id.isin(validation_set_utrip_ids)].sort_values(['utrip_id', 'checkin'])

We can now begin with data preprocessing.

We will combine trips into "sessions", discard trips that are too short and calculate total trip length.

We will use NVTabular for this work. It offers optimized tabular data preprocessing operators that run on the GPU. If you would like to learn more about the NVTabular library, please take a look [here](https://github.com/NVIDIA-Merlin/NVTabular).

In [5]:
train_set_dataset = Dataset(train_set)
validation_set_dataset = Dataset(validation_set)

In [9]:
weekday_checkin = (
    ["checkin"]
    >> ops.LambdaOp(lambda col: col.dt.weekday)
    >> ops.Rename(name="weekday_checkin")
)

weekday_checkout = (
    ["checkout"]
    >> ops.LambdaOp(lambda col: col.dt.weekday)
    >> ops.Rename(name="weekday_checkout")
)


groupby_features = ['city_id', 'booker_country', 'utrip_id', 'hotel_country', 'checkin'] + weekday_checkin + weekday_checkout >> ops.Groupby(
    groupby_cols=['utrip_id'],
    aggs={
        'city_id': ['list', 'count'],
        'booker_country': ['list'],
        'hotel_country': ['list'],
        'weekday_checkin': ['list'],
        'weekday_checkout': ['list']
    },
    sort_cols="checkin"
)

groupby_features_city = groupby_features['city_id_list'] >> ops.Categorify() >> ops.AddTags([Tags.SEQUENCE, Tags.ITEM, Tags.ITEM_ID])
groupby_features_country = (
    groupby_features['booker_country_list', 'hotel_country_list', 'weekday_checkin_list', 'weekday_checkout_list']
    >> ops.Categorify() >> ops.AddTags([Tags.SEQUENCE, Tags.ITEM])
)
city_id_count = groupby_features['city_id_count'] >> ops.AddTags([Tags.CONTEXT, Tags.ITEM, Tags.CONTINUOUS])

# Filter out sessions with less than 2 interactions 
MINIMUM_SESSION_LENGTH = 2
filtered_sessions = groupby_features_city + groupby_features_country + city_id_count >> ops.Filter(f=lambda df: df["city_id_count"] >= MINIMUM_SESSION_LENGTH) 

In [10]:
wf = Workflow(filtered_sessions)

In [11]:
train_set_processed = wf.fit_transform(train_set_dataset)
validation_set_processed = wf.transform(validation_set_dataset)

Our data consists of a sequence of visited `city_ids`, a sequence of `booker_countries` (represented as integer categories) and a `city_id_count` column (which contains the count of visited cities in a trip).

In [12]:
train_set_processed.compute().head()

city_id_list    booker_country_list  \
0      [8240, 156, 2278, 2097]           [3, 3, 3, 3]   
1      [63, 1160, 87, 619, 63]        [1, 1, 1, 1, 1]   
2  [7, 6, 24, 1051, 65, 52, 3]  [2, 2, 2, 2, 2, 2, 2]   
3          [1032, 757, 140, 3]           [2, 2, 2, 2]   
4   [3603, 262, 662, 250, 359]        [1, 1, 1, 1, 1]   

        hotel_country_list   weekday_checkin_list  weekday_checkout_list  \
0             [3, 3, 3, 3]           [5, 7, 4, 3]           [7, 4, 2, 7]   
1          [1, 1, 1, 1, 1]        [5, 1, 4, 3, 5]        [6, 4, 2, 5, 4]   
2  [2, 2, 2, 16, 16, 3, 3]  [5, 1, 2, 6, 5, 7, 4]  [6, 3, 1, 5, 7, 4, 3]   
3          [19, 19, 19, 3]           [1, 4, 2, 3]           [4, 3, 2, 5]   
4     [30, 30, 30, 30, 30]        [1, 3, 6, 5, 1]        [2, 1, 5, 6, 3]   

   city_id_count  
0              4  
1              5  
2              7  
3              4  
4              5

We are now ready to train our model.

In [13]:
import merlin.models.tf as mm

Let's identify two schemas. The first one for sequential features, the other for context features (`city_id_count`) that we will broadcast to the entire sequence.

Here is the schema of the data that our model will use.

In [14]:
seq_schema = train_set_processed.schema.select_by_tag(Tags.SEQUENCE)
seq_schema

[{'name': 'city_id_list', 'tags': {<Tags.ITEM_ID: 'item_id'>, <Tags.CATEGORICAL: 'categorical'>, <Tags.ID: 'id'>, <Tags.ITEM: 'item'>, <Tags.SEQUENCE: 'sequence'>}, 'properties': {'num_buckets': None, 'freq_threshold': 0, 'max_size': 0, 'start_index': 0, 'cat_path': './/categories/unique.city_id_list.parquet', 'domain': {'min': 0, 'max': 37202, 'name': 'city_id_list'}, 'embedding_sizes': {'cardinality': 37203, 'dimension': 512}}, 'dtype': dtype('int64'), 'is_list': True, 'is_ragged': True}, {'name': 'booker_country_list', 'tags': {<Tags.SEQUENCE: 'sequence'>, <Tags.CATEGORICAL: 'categorical'>, <Tags.ITEM: 'item'>}, 'properties': {'num_buckets': None, 'freq_threshold': 0, 'max_size': 0, 'start_index': 0, 'cat_path': './/categories/unique.booker_country_list.parquet', 'domain': {'min': 0, 'max': 5, 'name': 'booker_country_list'}, 'embedding_sizes': {'cardinality': 6, 'dimension': 16}}, 'dtype': dtype('int64'), 'is_list': True, 'is_ragged': True}, {'name': 'hotel_country_list', 'tags': {<Tags.SEQUENCE: 'sequence'>, <Tags.CATEGORICAL: 'categorical'>, <Tags.ITEM: 'item'>}, 'properties': {'num_buckets': None, 'freq_threshold': 0, 'max_size': 0, 'start_index': 0, 'cat_path': './/categories/unique.hotel_country_list.parquet', 'domain': {'min': 0, 'max': 194, 'name': 'hotel_country_list'}, 'embedding_sizes': {'cardinality': 195, 'dimension': 31}}, 'dtype': dtype('int64'), 'is_list': True, 'is_ragged': True}, {'name': 'weekday_checkin_list', 'tags': {<Tags.SEQUENCE: 'sequence'>, <Tags.CATEGORICAL: 'categorical'>, <Tags.ITEM: 'item'>}, 'properties': {'num_buckets': None, 'freq_threshold': 0, 'max_size': 0, 'start_index': 0, 'cat_path': './/categories/unique.weekday_checkin_list.parquet', 'domain': {'min': 0, 'max': 7, 'name': 'weekday_checkin_list'}, 'embedding_sizes': {'cardinality': 8, 'dimension': 16}}, 'dtype': dtype('int64'), 'is_list': True, 'is_ragged': True}, {'name': 'weekday_checkout_list', 'tags': {<Tags.SEQUENCE: 'sequence'>, <Tags.CATEGORICAL: 'categorical'>, <Tags.ITEM: 'item'>}, 'properties': {'num_buckets': None, 'freq_threshold': 0, 'max_size': 0, 'start_index': 0, 'cat_path': './/categories/unique.weekday_checkout_list.parquet', 'domain': {'min': 0, 'max': 7, 'name': 'weekday_checkout_list'}, 'embedding_sizes': {'cardinality': 8, 'dimension': 16}}, 'dtype': dtype('int64'), 'is_list': True, 'is_ragged': True}]

Let's also identify the target column.

In [15]:
target = train_set_processed.schema.select_by_tag(Tags.SEQUENCE).column_names[0]
target

'city_id_list'

## Constructing the model

We begin by defining our `Loader`. It will be responsible for batching our data and passing it to the model.

In [16]:
loader = Loader(train_set_processed, batch_size=64, shuffle=True)

And now onto model construction.

We can specify various hyperparameters, such as the number of heads and number of layers to use.

For the transformer portion of our model, we will use the `XLNet` architecture. Additionally, we are passing `mm.ReplaceMaskedEmbeddings()` as our `pre` block. We will be training a masked language model and this parameter is responsible for the masking of our sequences.

Later, when we run the `fit` method on our model, we will specify the `masking_probability` of `0.3`. Through the combination of these parameters, our model will train on sequences where any given timestep will be masked with a probability of 0.3 and it will be our model's training task to infer the target value for that step!

To summarize, Masked Language Modeling is implemented by using two blocks in combination:

* `SequenceMaskRandom()` - Used as a pre for `model.fit()`, it randomly selects items from the sequence to be masked for prediction as targets, by using Keras masking.
* `ReplaceMaskedEmbeddings()` - Used as a pre for a `TransformerBlock`, it replaces the input embeddings at masked positions for prediction by a dummy trainable embedding, to avoid leakage of the targets.

In [17]:
model = mm.Model(
    mm.InputBlockV2(
        seq_schema,
        embeddings=mm.Embeddings(
            train_set_processed.schema.select_by_tag(Tags.CATEGORICAL), sequence_combiner=None
        ),
    ),
    mm.XLNetBlock(d_model=64, n_head=4, n_layer=2, pre=mm.ReplaceMaskedEmbeddings()),
    mm.CategoricalOutput(
        train_set_processed.schema.select_by_name(target),
        default_loss="categorical_crossentropy",
    ),
)

## Model training

In [18]:
model.compile(run_eagerly=False, optimizer='adam', loss="categorical_crossentropy")
model.fit(loader, epochs=5, pre=mm.SequenceMaskRandom(schema=seq_schema, target=target, masking_prob=0.3))

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


Epoch 1/5


2022-11-16 02:16:54.558116: I tensorflow/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8500


/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/xl_net_block/prepare_transformer_inputs_4/RaggedToTensor/boolean_mask_1/GatherV2:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model/xl_net_block/prepare_transformer_inputs_4/RaggedToTensor/boolean_mask/GatherV2:0", shape=(None, 64), dtype=float32), dense_shape=Tensor("gradient_tape/model/xl_net_block/prepare_transformer_inputs_4/RaggedToTensor/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/xl_net_block/replace_masked_embeddings/RaggedWhere/Reshape_3:0", shape=(None,), dtype=int64), values=Tensor("gradient_tape/model/xl_ne

2022-11-16 02:17:02.465694: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: model/xl_net_block/replace_masked_embeddings/RaggedWhere/Assert/AssertGuard/branch_executed/_9
/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


2720/2720 [==============================] - 113s 37ms/step - loss: 0.9611 - recall_at_10: 0.0930 - mrr_at_10: 0.0420 - ndcg_at_10: 0.0540 - map_at_10: 0.0420 - precision_at_10: 0.0093 - regularization_loss: 0.0000e+00 - loss_batch: 0.9612
Epoch 2/5
2720/2720 [==============================] - 106s 37ms/step - loss: 0.7824 - recall_at_10: 0.1500 - mrr_at_10: 0.0710 - ndcg_at_10: 0.0896 - map_at_10: 0.0710 - precision_at_10: 0.0150 - regularization_loss: 0.0000e+00 - loss_batch: 0.7824
Epoch 3/5
2720/2720 [==============================] - 106s 38ms/step - loss: 0.7318 - recall_at_10: 0.1602 - mrr_at_10: 0.0765 - ndcg_at_10: 0.0962 - map_at_10: 0.0765 - precision_at_10: 0.0160 - regularization_loss: 0.0000e+00 - loss_batch: 0.7319
Epoch 4/5
2720/2720 [==============================] - 106s 37ms/step - loss: 0.7130 - recall_at_10: 0.1691 - mrr_at_10: 0.0812 - ndcg_at_10: 0.1019 - map_at_10: 0.0812 - precision_at_10: 0.0169 - regularization_loss: 0.0000e+00 - loss_batch: 0.7131
Epoch 5/5


## Model evaluation

We have trained our model.

But in training the metrics come from a masked language modelling task. A portion of steps in the sequence was masked for each example. The metrics were calculated on this task.

In reality, we probably care how well our model does on the next item prediction task (as it mimics the scenario in which the model would be likely to be used).

Let's measure the performance of the model on a task where it attempts to predict the last item in a sequence.

We will mask the last item using `SequenceMaskLast` and run inference.

In [19]:
loader_eval = Loader(validation_set_processed, batch_size=128, shuffle=False)

In [20]:
model.evaluate(loader_eval, batch_size=128, pre=mm.SequenceMaskLast(schema=validation_set_processed.schema, target=target))

2022-11-16 02:25:57.739742: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: model/xl_net_block/replace_masked_embeddings/RaggedWhere/Assert/AssertGuard/branch_executed/_9


340/340 [==============================] - 21s 52ms/step - loss: 0.3228 - recall_at_10: 0.5691 - mrr_at_10: 0.3137 - ndcg_at_10: 0.3746 - map_at_10: 0.3137 - precision_at_10: 0.0569 - regularization_loss: 0.0000e+00 - loss_batch: 0.3227


[0.3228282928466797,
 0.5691077709197998,
 0.31373801827430725,
 0.3745550513267517,
 0.31373801827430725,
 0.05691079422831535,
 0.2738809883594513,
 0.0]